# Exploring AI & ML Job Trends in the U.S.

# Notebook Version: v1  
**Focus**: Dataset loading and basic structural preview  

This notebook is part of a versioned project exploring trends in AI/ML job postings in the U.S.  
This version focuses on loading the dataset, checking its structure, and identifying surface-level issues.


In [ ]:
#importing the necessary libraries
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

## Dataset Overview

- Source: Kaggle – AI and ML Job Listings USA  
- File path: `/kaggle/input/ai-and-ml-job-listings-usa/ai_ml_jobs_linkedin.csv

## Load and Preview Data

Loading the dataset into a DataFrame and preview the structure to understand its basic layout.


In [ ]:
# Load the dataset
us_jobs_df = pd.read_csv('/kaggle/input/ai-and-ml-job-listings-usa/ai_ml_jobs_linkedin.csv')

# Create a working copy
jobs_df = us_jobs_df.copy()

In [ ]:
# Preview first 2 rows
jobs_df.head(2)


In [ ]:
# Check dataset shape
print(f"Rows: {jobs_df.shape[0]}, Columns: {jobs_df.shape[1]}")

# Data types and non-null info
jobs_df.info()

In [ ]:
# Summary stats for numeric columns
jobs_df.describe()

## Initial Observations and Notes

- The dataset contains **862 rows** and **10 columns**.
- Some columns such as `companyName`, `publishedAt`, and `sector` contain missing values.
- Columns like `applicationsCount` and `publishedAt` may need data type conversions in the next version.
- No immediate data loading issues were encountered.


# Notebook Version: v2  
**Focus**: Data Cleaning and Formatting  
 
This version focuses on cleaning the dataset, handling missing values, renaming columns, correcting data types, and preparing the data for analysis.


In [ ]:
#previewing the data again
jobs_df.head(3)

## Handling missing/null values

In [ ]:
#checking for null values if any
jobs_df.isna().sum()

## Data Type Fix

In [ ]:
#handling null values

#filling the null values in columns 'companyName' and 'experienceLevel' as 'Unknown'
jobs_df[['companyName','sector']] = jobs_df[['companyName','sector']].fillna('Unknown')

#handling the null value for column 'publishedAt' using ffill() assuming that the post has been updated nearly at that date
jobs_df['publishedAt'] = jobs_df['publishedAt'].fillna(method='ffill')

#check if null value still exists
jobs_df.isna().sum()


#### NOTE:
Filled publishedAt using forward fill to maintain temporal continuity, assuming listings are updated close to previous records.

In [ ]:
#checking for the datatypes 
jobs_df.info()

In [ ]:
# converting 'publishedAt' into datetime data type  
jobs_df['publishedAt'] = pd.to_datetime(jobs_df['publishedAt'])

# converting 'applicationsCount' into integer data type
#first we need to extract the count of the applications 
jobs_df['applicationsCount'] = jobs_df['applicationsCount'].str.extract(r'(\d+)')[0]

#now convert the 'applicationsCount' dtype to numeric
jobs_df['applicationsCount'] = pd.to_numeric(jobs_df['applicationsCount'])
jobs_df.info()


## Removing the columns that are not useful for my analysis

In [ ]:
#making a new df to store only the columns that are useful for my analysis

updated_jobs = jobs_df.drop(columns=['description','sector','workType'])
updated_jobs

## Duplicate Check and Removal

In [ ]:
# checking the duplicate values that exists (based on all columns)
# updated_jobs.duplicated().sum()
updated_jobs[updated_jobs.duplicated()]

# dropping the duplicated values
updated_jobs.drop_duplicates(inplace=True)

# check if any row exist that have same title,companyName, location and publishedAt
updated_jobs.duplicated(subset=['title', 'companyName', 'location', 'publishedAt']).sum()

# removing the dupliacted values
duplicate_vals = updated_jobs.duplicated(subset=['title', 'companyName', 'location', 'publishedAt'])
updated_jobs = updated_jobs[~duplicate_vals].copy()

#resetting the index after droppingt the duplicate values
updated_jobs.reset_index(drop=True,inplace=True)

## Whitespace Stripping 



In [ ]:
#stripping the whitespaces if any, from the string based columns

for col in ['title','companyName','location','experienceLevel','contractType']:
    updated_jobs[col] = updated_jobs[col].str.strip()



## Category Cleaning

In [ ]:
#1. title
# for consistency I am converting the titles into title case
updated_jobs['title'] = updated_jobs['title'].str.title()

#check if the function is applied properly
updated_jobs['title'].head(3)

#2. location
# I will be splitting the location column into two parts: one is for city and other is for state
location_split = updated_jobs['location'].str.split(',',n=1,expand=True)

#adding the city column
updated_jobs['city'] = location_split[0].str.strip()

#adding the state column
updated_jobs['state'] = location_split[1].str.strip()

#check if any null value has been added due to the above two columns
updated_jobs.isna().sum()

#handle the null values
updated_jobs['state'] = updated_jobs['state'].fillna('Unknown')

#rechecking for null values
updated_jobs.isna().sum()

#removing the location column as it is no more useful
updated_jobs.drop('location',axis='columns',inplace=True)

#3. publishedAt
# I will be asplitting this column also into two parts year and month (day is not useful)

updated_jobs['year'] = updated_jobs['publishedAt'].dt.year
updated_jobs['month'] = updated_jobs['publishedAt'].dt.month

#dropping the publishedAt column because it is not useful
updated_jobs.drop('publishedAt',axis='columns',inplace=True)
updated_jobs.columns

#4. companyName
# converting the company's name into title case so that it remains consistent throughout
updated_jobs['companyName'] = updated_jobs['companyName'].str.title()

#check if the change has been made properly
updated_jobs['companyName'].head(5)

## Dataset Cleaning and Structuring Summary

In this version, I focused on cleaning and structuring the dataset to prepare it for meaningful analysis. The original dataset had multiple inconsistencies and mixed-format fields which could hinder exploration and insights.

## 🚀 Key actions
- Selected 7 relevant columns for the analysis.
- Cleaned categorical columns (`title`, `companyName`, `location`) for consistency.
- Split complex fields like `location` and `publishedAt` into simpler, analyzable components (city, state, year, month).
- Handled missing values in `state` by filling with "Unknown".

## ⚠️ Challenges
- Some job titles were overly specific or inconsistent (e.g., different casing, role modifiers). I resolved this with title casing but might need more grouping later.
- The `location` field didn’t follow a uniform format in all rows — some were missing state info, which led to NaN values after splitting.
- The `publishedAt` field contained full timestamps, which were not useful at this stage. It took care to isolate only the useful components (year/month) without losing meaning.

## 🎯 Learnings
- Even basic string cleaning and formatting (like `.str.title()` or `.str.strip()`) can greatly improve consistency in the dataset.
- Breaking down complex columns (like `location` and `publishedAt`) can make future analysis smoother and more insightful.
- It's important to analyze columns one by one instead of applying generic cleaning — each column may need unique handling.



## 